In [1]:
%load_ext autoreload
%autoreload 2

from fastai.text.all import *
from fastai.vision.all import *
import pandas as pd
import torch

In [2]:
OUT_DIM=6

In [3]:
seed = 42

# python RNG
import random
random.seed(seed)

# pytorch RNGs
import torch
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)

# numpy RNG
import numpy as np
np.random.seed(seed)

In [5]:
path = Path("./data")

In [6]:
files = get_files(path); files

(#359882) [Path('data/valid/agravo_em_recurso_extraordinario/ARE_957523_309003212_1420_21032016_5.activations'),Path('data/valid/agravo_em_recurso_extraordinario/ARE_811832_4448371_1420_18052014_17.activations'),Path('data/valid/agravo_em_recurso_extraordinario/ARE_1053618_311977154_1420_07062017_63.activations'),Path('data/valid/agravo_em_recurso_extraordinario/ARE_1066591_312429859_1420_11082017_9.activations'),Path('data/valid/agravo_em_recurso_extraordinario/ARE_1071533_312604954_1420_30082017_6.activations'),Path('data/valid/agravo_em_recurso_extraordinario/ARE_1105199_313609724_1420_04022018_13.activations'),Path('data/valid/agravo_em_recurso_extraordinario/ARE_1010513_310734552_1420_07022017_5.activations'),Path('data/valid/agravo_em_recurso_extraordinario/ARE_1056724_312077006_1420_27062017_10.activations'),Path('data/valid/agravo_em_recurso_extraordinario/ARE_1115582_313936746_1420_16032018_5.activations'),Path('data/valid/agravo_em_recurso_extraordinario/ARE_1101826_313537521

In [66]:
class TensorToTorch(Transform):
    def encodes(self, x):
        with open(x, "rb") as file:
            t = pickle.load(file)
        return tensor(np.array(t)).view(-1)

In [67]:
tfms = [TensorToTorch(),[parent_label, Categorize()]]
splits = GrandparentSplitter(valid_name='valid')(files)
dsets = Datasets(files, tfms, splits=splits)
dls = dsets.dataloaders()

In [68]:
dls.bs

64

In [69]:
test_splits = GrandparentSplitter(valid_name='test')(files)
test_dsets = Datasets(files, tfms, splits=test_splits)
test_dls = test_dsets.dataloaders()

In [70]:
class ImgTextFusion(Module):
    def __init__(self, head):
        self.head = head
    
    def forward(self, x):
        return self.head(x)

In [71]:
def create_head(nf, n_out, lin_ftrs=None, ps=0.5, bn_final=False, lin_first=False):
    "Model head that takes `nf` features, runs through `lin_ftrs`, and out `n_out` classes."
    lin_ftrs = [nf, 512, n_out] if lin_ftrs is None else [nf] + lin_ftrs + [n_out]
    ps = L(ps)
    if len(ps) == 1: ps = [ps[0]/2] * (len(lin_ftrs)-2) + ps
    actns = [nn.ReLU(inplace=True)] * (len(lin_ftrs)-2) + [None]
    layers = []
    if lin_first: layers.append(nn.Dropout(ps.pop(0)))
    for ni,no,p,actn in zip(lin_ftrs[:-1], lin_ftrs[1:], ps, actns):
        layers += LinBnDrop(ni, no, bn=True, p=p, act=actn, lin_first=lin_first)
    if lin_first: layers.append(nn.Linear(lin_ftrs[-2], n_out))
    if bn_final: layers.append(nn.BatchNorm1d(lin_ftrs[-1], momentum=0.01))
    return nn.Sequential(*layers)

In [72]:
head = create_head(12, OUT_DIM)

In [73]:
model = ImgTextFusion(head)

In [74]:
model

ImgTextFusion(
  (head): Sequential(
    (0): BatchNorm1d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): Dropout(p=0.25, inplace=False)
    (2): Linear(in_features=12, out_features=512, bias=False)
    (3): ReLU(inplace=True)
    (4): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): Dropout(p=0.5, inplace=False)
    (6): Linear(in_features=512, out_features=6, bias=False)
  )
)

In [75]:
def loss_func(out, targ):
    return CrossEntropyLossFlat()(out, targ.long())

In [76]:
fscore = F1Score(average='weighted')

In [77]:
learn = Learner(dls, model, loss_func=loss_func, metrics=[accuracy, fscore])

In [78]:
dls.one_batch()[0].shape

torch.Size([64, 12])

In [79]:
learn.model(dls.one_batch()[0])

RuntimeError: Tensor for argument #2 'weight' is on CPU, but expected it to be on GPU (while checking arguments for cudnn_batch_norm)

In [60]:
learn.lr_find()

KeyboardInterrupt: 

In [ ]:
learn.fit_one_cycle(10, 3e-3)